In [1]:
from ur5 import UR5
from math import pi
ur5 = UR5()

Inicializando a classe UR5...
Inicializando os nós e handles...
Pronto!


In [3]:
ur5.actuate_gripper(close=0)

Iterações totais:  63
2.016


In [2]:
ur5.move_to_config([0, 0, 0, 0, 0, 0])

Iterações totais:  157


(5.024, 0.1360575231665497, 0.03294947782686581)

In [7]:
ur5.move_to_config([.5*pi, .05*pi, .45*pi, 0, -.5*pi, .5*pi])

Iterações totais:  157


(5.024000000000001, 0.10390928968378059, 0.032766049589412795)

In [17]:
ur5.move_to_config([.4*pi, .18*pi, .57*pi, 0.25*pi, 0.8, 0])

Iterações totais:  157


(5.024000000000001, 0.1356689656504644, 0.033869113792508634)

In [ ]:
# bottle = supervisor.getFromDef('bottle')

In [ ]:
# bottle.getField('translation').setSFVec3f([-1.467559345817077, 1.7836801495521285e-06, 0.9004484925267724])

In [ ]:
# bottle.getField('translation').getSFVec3f()